## Import modules and tools

In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from os import path
from os import listdir
import datetime, time

In [2]:
# costome codes

import ma_codes as codes
from ma_codes import mysize, mymap

#### Define a nice colour map for gene expression

In [3]:
from matplotlib import colors
colorsComb = np.vstack([plt.cm.Reds(np.linspace(0, 1, 128)), plt.cm.Greys_r(np.linspace(0.7, 0.8, 0))])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

In [4]:
plt.rcParams['savefig.dpi'] = 200 #图片像素
plt.rcParams['figure.dpi'] = 200 #分辨率

In [5]:
import pandas as pd
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

#### set timestamp function

In [6]:
def timestamp():
    ts = time.time()
    st = datetime.datetime.fromtimFestamp(ts).strftime('%d-%m-%Y %H:%M:%S')
    return st

#### workdirectory

In [7]:
folder = "/media/qianjianghu/SG_QJData/DataAnlysis/Jupyter/WNT_PCLS/"

#### scanpy setting and version

In [8]:
sc.settings.verbosity = 4             # verbosity: 0=’error’, 1=’warning’, 2=’info’, 3=’hint’, 4=more details, 5=even more details, etc.
sc.logging.print_header()
Obj_folder = '/media/qianjianghu/SG_QJData/DataAnlysis/Jupyter/WNT_PCLS/Object_folder/'  # the file that will store the analysis results

scanpy==1.6.0 anndata==0.7.5 umap==0.4.6 numpy==1.18.5 scipy==1.5.2 pandas==1.1.5 scikit-learn==0.23.2 statsmodels==0.12.0 python-igraph==0.8.3 louvain==0.6.1 leidenalg==0.8.3


## Reload adata

In [9]:
EPITHELIAL = sc.read(Obj_folder + "/PP5_EPITHELIAL_cluster_WNT_PCLS.h5ad")

## ranking genes each cluster

In [10]:
EPITHELIAL

AnnData object with n_obs × n_vars = 4343 × 23558
    obs: 'project', 'genome', 'groups', 'treatment', 'treated_days', 'estimated_cells', 'identifier', 'doublet_scores', 'predicted_doublets', 'n_counts', 'n_genes', 'percent.mito', 'size_factors', 'S_score', 'G2M_score', 'phase', 'louvain', 'leiden', 'leiden_0.3', 'TYPE', 'Epi_leiden_2', 'cell_type', 'Epi_leiden_4', 'Epi_leiden_5', 'Epi_leiden_6', 'cell_type_new', 'cluster'
    var: 'n_cells', 'highly_variable', 'mean', 'std'
    uns: 'Epi_leiden_2_colors', 'Epi_leiden_4_colors', 'Epi_leiden_5_colors', 'Epi_leiden_6_colors', 'TYPE_colors', 'cell_type_colors', 'cell_type_new_colors', 'cluster_colors', 'groups_colors', 'identifier_colors', 'leiden', 'leiden_colors', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'rank_genes_cluster', 'rank_genes_groups', 'treated_days_colors', 'treatment_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [11]:
#Calculate marker genes
sc.tl.rank_genes_groups(EPITHELIAL, groupby='cell_type_new', key_added='rank_genes_cluster_cell_type', n_genes=500)

ranking genes
    consider 'cell_type_new' groups:
    with sizes: [403 346 329 325 309 174 232 210 203 185 181 180 161 160 121 110 102  92
  89  85  65  62  60  27  17 115]
    finished: added to `.uns['rank_genes_cluster_cell_type']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:07)


In [12]:
result_folder = '/media/qianjianghu/SG_QJData/DataAnlysis/Jupyter/WNT_PCLS/results/'

#### export and layout clusters horizontally

In [13]:
####
#### layout clusters horizontally
####

result = EPITHELIAL.uns['rank_genes_cluster_cell_type']
groups = result['names'].dtype.names
cluster_marker = pd.DataFrame(
    {'Cluster'+group + '_' + key: result[key][group]
    for group in groups for key in ['names', 'pvals','pvals_adj', 'logfoldchanges']})

In [14]:
cluster_marker.head()

,ClusterC0_names,ClusterC0_pvals,ClusterC0_pvals_adj,ClusterC0_logfoldchanges,ClusterC1_names,ClusterC1_pvals,ClusterC1_pvals_adj,ClusterC1_logfoldchanges,ClusterC2_Krt8_ADI_names,ClusterC2_Krt8_ADI_pvals,ClusterC2_Krt8_ADI_pvals_adj,ClusterC2_Krt8_ADI_logfoldchanges,ClusterC3_ciliated_names,ClusterC3_ciliated_pvals,ClusterC3_ciliated_pvals_adj,ClusterC3_ciliated_logfoldchanges,ClusterC4_basal_names,ClusterC4_basal_pvals,ClusterC4_basal_pvals_adj,ClusterC4_basal_logfoldchanges,ClusterC5_names,ClusterC5_pvals,ClusterC5_pvals_adj,ClusterC5_logfoldchanges,ClusterC6_ciliated_names,ClusterC6_ciliated_pvals,ClusterC6_ciliated_pvals_adj,ClusterC6_ciliated_logfoldchanges,ClusterC7_names,ClusterC7_pvals,ClusterC7_pvals_adj,ClusterC7_logfoldchanges,ClusterC8_basal_names,ClusterC8_basal_pvals,ClusterC8_basal_pvals_adj,ClusterC8_basal_logfoldchanges,ClusterC9_names,ClusterC9_pvals,ClusterC9_pvals_adj,ClusterC9_logfoldchanges,ClusterC10_ciliated_names,ClusterC10_ciliated_pvals,ClusterC10_ciliated_pvals_adj,ClusterC10_ciliated_logfoldchanges,ClusterC11_Club_names,ClusterC11_Club_pvals,ClusterC11_Club_pvals_adj,ClusterC11_Club_logfoldchanges,ClusterC12_AT2_names,ClusterC12_AT2_pvals,ClusterC12_AT2_pvals_adj,ClusterC12_AT2_logfoldchanges,ClusterC13_Club_names,ClusterC13_Club_pvals,ClusterC13_Club_pvals_adj,ClusterC13_Club_logfoldchanges,ClusterC14_ciliated_names,ClusterC14_ciliated_pvals,ClusterC14_ciliated_pvals_adj,ClusterC14_ciliated_logfoldchanges,ClusterC15_ciliated_names,ClusterC15_ciliated_pvals,ClusterC15_ciliated_pvals_adj,ClusterC15_ciliated_logfoldchanges,ClusterC16_names,ClusterC16_pvals,ClusterC16_pvals_adj,ClusterC16_logfoldchanges,ClusterC17_names,ClusterC17_pvals,ClusterC17_pvals_adj,ClusterC17_logfoldchanges,ClusterC18_AT2_names,ClusterC18_AT2_pvals,ClusterC18_AT2_pvals_adj,ClusterC18_AT2_logfoldchanges,ClusterC19_club_names,ClusterC19_club_pvals,ClusterC19_club_pvals_adj,ClusterC19_club_logfoldchanges,ClusterC20_club_names,ClusterC20_club_pvals,ClusterC20_club_pvals_adj,ClusterC20_club_logfoldchanges,ClusterC21_krt8High_names,ClusterC21_krt8High_pvals,ClusterC21_krt8High_pvals_adj,ClusterC21_krt8High_logfoldchanges,ClusterC22_krt8high_names,ClusterC22_krt8high_pvals,ClusterC22_krt8high_pvals_adj,ClusterC22_krt8high_logfoldchanges,ClusterC23_neuroendocrine_names,ClusterC23_neuroendocrine_pvals,ClusterC23_neuroendocrine_pvals_adj,ClusterC23_neuroendocrine_logfoldchanges,ClusterC24_AT1_names,ClusterC24_AT1_pvals,ClusterC24_AT1_pvals_adj,ClusterC24_AT1_logfoldchanges,ClusterC25_Krt8_ADI_2_names,ClusterC25_Krt8_ADI_2_pvals,ClusterC25_Krt8_ADI_2_pvals_adj,ClusterC25_Krt8_ADI_2_logfoldchanges
0,mt-Rnr2,1.815628e-265,4.277256e-261,1.852911,mt-Rnr2,1.077940e-204,2.539411e-200,1.925961,Tmsb4x,1.686682e-100,5.676408e-97,2.237518,Nupr1,8.588919e-68,3.372296e-64,2.806788,mt-Rnr2,3.511470e-85,1.034040e-81,1.356172,Clu,4.028291e-38,1.216647e-35,2.546970,Tmem212,1.036238e-94,4.068617e-91,4.224972,Esd,2.414016e-35,1.263764e-32,1.677655,Fth1,4.946481e-42,1.942153e-39,1.656945,Clu,1.914625e-29,1.804190e-27,2.550858,Fth1,2.124783e-100,5.561736e-97,1.985962,Trf,4.815077e-58,5.401599e-55,5.199295,Sftpc,2.364060e-257,5.569252e-253,9.878492,Selenbp1,2.759313e-115,1.625098e-111,4.938071,Sec14l3,5.204444e-78,6.811460e-75,5.871947,Tm4sf1,8.191721e-31,2.412257e-27,2.738625,Hnrnpc,0.000001,0.000011,1.679872,Ccdc153,1.120141e-10,3.340290e-09,2.646315,Sftpc,2.947855e-56,7.548430e-54,8.415749,Wfdc2,1.140740e-34,2.465463e-32,2.918976,Scgb1a1,5.703205e-61,1.119634e-57,7.746153,Krt8,1.363841e-37,5.841704e-35,3.100900,mt-Rnr2,2.797602e-34,6.046413e-33,2.024664,Resp18,0.000021,0.000121,11.269704,Ager,6.861547e-14,3.195183e-13,7.902991,Clu,3.478355e-39,7.730481e-37,2.777132
1,Malat1,4.415912e-83,3.467668e-80,1.667595,Malat1,9.391336e-128,5.531027e-124,1.999551,Krt8,1.954261e-70,1.705129e-67,2.396628,Fth1,1.508342e-47,1.776676e-44,1.326333,Rps12,1.221775e-31,8.591810e-30,1.853010,Cldn4,2.435506e-24,3.984420e-22,2.438120,Ccdc153,1.096727e-87,2.583669e-84,3.873407,

In [15]:
cluster_marker.tail()

,ClusterC0_names,ClusterC0_pvals,ClusterC0_pvals_adj,ClusterC0_logfoldchanges,ClusterC1_names,ClusterC1_pvals,ClusterC1_pvals_adj,ClusterC1_logfoldchanges,ClusterC2_Krt8_ADI_names,ClusterC2_Krt8_ADI_pvals,ClusterC2_Krt8_ADI_pvals_adj,ClusterC2_Krt8_ADI_logfoldchanges,ClusterC3_ciliated_names,ClusterC3_ciliated_pvals,ClusterC3_ciliated_pvals_adj,ClusterC3_ciliated_logfoldchanges,ClusterC4_basal_names,ClusterC4_basal_pvals,ClusterC4_basal_pvals_adj,ClusterC4_basal_logfoldchanges,ClusterC5_names,ClusterC5_pvals,ClusterC5_pvals_adj,ClusterC5_logfoldchanges,ClusterC6_ciliated_names,ClusterC6_ciliated_pvals,ClusterC6_ciliated_pvals_adj,ClusterC6_ciliated_logfoldchanges,ClusterC7_names,ClusterC7_pvals,ClusterC7_pvals_adj,ClusterC7_logfoldchanges,ClusterC8_basal_names,ClusterC8_basal_pvals,ClusterC8_basal_pvals_adj,ClusterC8_basal_logfoldchanges,ClusterC9_names,ClusterC9_pvals,ClusterC9_pvals_adj,ClusterC9_logfoldchanges,ClusterC10_ciliated_names,ClusterC10_ciliated_pvals,ClusterC10_ciliated_pvals_adj,ClusterC10_ciliated_logfoldchanges,ClusterC11_Club_names,ClusterC11_Club_pvals,ClusterC11_Club_pvals_adj,ClusterC11_Club_logfoldchanges,ClusterC12_AT2_names,ClusterC12_AT2_pvals,ClusterC12_AT2_pvals_adj,ClusterC12_AT2_logfoldchanges,ClusterC13_Club_names,ClusterC13_Club_pvals,ClusterC13_Club_pvals_adj,ClusterC13_Club_logfoldchanges,ClusterC14_ciliated_names,ClusterC14_ciliated_pvals,ClusterC14_ciliated_pvals_adj,ClusterC14_ciliated_logfoldchanges,ClusterC15_ciliated_names,ClusterC15_ciliated_pvals,ClusterC15_ciliated_pvals_adj,ClusterC15_ciliated_logfoldchanges,ClusterC16_names,ClusterC16_pvals,ClusterC16_pvals_adj,ClusterC16_logfoldchanges,ClusterC17_names,ClusterC17_pvals,ClusterC17_pvals_adj,ClusterC17_logfoldchanges,ClusterC18_AT2_names,ClusterC18_AT2_pvals,ClusterC18_AT2_pvals_adj,ClusterC18_AT2_logfoldchanges,ClusterC19_club_names,ClusterC19_club_pvals,ClusterC19_club_pvals_adj,ClusterC19_club_logfoldchanges,ClusterC20_club_names,ClusterC20_club_pvals,ClusterC20_club_pvals_adj,ClusterC20_club_logfoldchanges,ClusterC21_krt8High_names,ClusterC21_krt8High_pvals,ClusterC21_krt8High_pvals_adj,ClusterC21_krt8High_logfoldchanges,ClusterC22_krt8high_names,ClusterC22_krt8high_pvals,ClusterC22_krt8high_pvals_adj,ClusterC22_krt8high_logfoldchanges,ClusterC23_neuroendocrine_names,ClusterC23_neuroendocrine_pvals,ClusterC23_neuroendocrine_pvals_adj,ClusterC23_neuroendocrine_logfoldchanges,ClusterC24_AT1_names,ClusterC24_AT1_pvals,ClusterC24_AT1_pvals_adj,ClusterC24_AT1_logfoldchanges,ClusterC25_Krt8_ADI_2_names,ClusterC25_Krt8_ADI_2_pvals,ClusterC25_Krt8_ADI_2_pvals_adj,ClusterC25_Krt8_ADI_2_logfoldchanges
495,Arnt,0.032566,0.093396,1.832983,Tsc22d4,0.009018,0.036542,1.184173,Eif3b,0.041997,0.095150,0.447516,C230072F16Rik,0.041947,0.128004,1.664326,Tomm6,0.290014,0.418366,1.381117,Bin1,0.000780,0.009347,1.074733,mt-Nd1,4.172419e-13,1.136345e-11,0.875670,Dstn,0.000002,0.000046,0.791598,Abce1,0.183354,0.359744,0.678382,Pard3b,0.153584,0.315361,1.434699,Arl6,3.281982e-07,0.000007,1.922322,Lman2,0.119028,0.302619,0.715545,Rpl36a,0.000952,0.006260,0.733407,Atxn7l3b,3.212953e-07,0.000006,1.100454,Selenok,0.000007,0.000090,1.407580,Drc7,0.000508,0.008692,1.459904,Mgat2,0.096119,0.257725,1.133836,Psmc3ip,0.080204,0.243538,1.330889,Sltm,0.002328,0.010999,1.111825,Abcd3,0.007260,0.047893,0.967750,Wdr83os,0.052565,0.186045,1.388251,Cpsf6,0.000317,0.002985,2.129710,Hmg20b,0.183082,0.328064,1.248404,Oga,0.153543,0.321771,1.272359,Cpq,0.293007,0.409022,3.042598,Sdcbp2,0.000873,0.008770,1.816639
496,Chd1,0.032681,0.093650,0.728637,Phf11b,0.009076,0.036756,3.905086,Dhrs1,0.042406,0.095947,0.906173,Klhl12,0.043011,0.130405,1.282985,Hypk,0.290728,0.418366,0.531313,Irf2bpl,0.000788,0.009419,1.268461,2010001K21Rik,6.155210e-13,1.651531e-11,3.249036,Clic4,0.000002,0.000045,0.806389,Capg,0.183416,0.359775,0.586780,Lzts2,0.153567,0.315361,0.951293,Cspp1,3.196499e-07,0.000007,1.274906,Fgfr1,0.119565,0.303621,1.901690,Npr1,0.000990,0.006484,2.954659,Wdr45,3.437866e-07,0

In [16]:
cluster_marker.to_csv(result_folder+"EPITHELIAL_WNT_PCLS_cluster_marker_horizontally.csv")

#### export and layout clusters vertically

In [17]:
####
#### layout clusters vertically
####

cluster_marker=pd.DataFrame(columns = ['names', 'scores','logfoldchanges','pvals','pvals_adj','cluster'])
for group in groups:
    group_marker = pd.DataFrame(
    {key: result[key][group]
    for key in ['names', 'scores','logfoldchanges','pvals','pvals_adj']})
    group_marker['cluster'] = group
    cluster_marker=cluster_marker.append(group_marker)                                    

In [18]:
cluster_marker.head()

,names,scores,logfoldchanges,pvals,pvals_adj,cluster
0,mt-Rnr2,45.305584,1.852911,1.815628e-265,4.277256e-261,C0
1,Malat1,22.482218,1.667595,4.415912e-83,3.467668e-80,C0
2,mt-Rnr1,13.649235,1.311597,3.012854e-36,2.628771e-34,C0
3,Gm8995,11.309280,3.179454,4.782000e-26,2.213248e-24,C0
4,Akap12,8.949334,2.696359,1.176642e-17,2.828503e-16,C0


In [19]:
cluster_marker.tail()

,names,scores,logfoldchanges,pvals,pvals_adj,cluster
495,Sdcbp2,3.416520,1.816639,0.000873,0.008770,C25_Krt8_ADI_2
496,Cnih4,3.416517,1.192521,0.000866,0.008713,C25_Krt8_ADI_2
497,Atp13a1,3.408000,2.741052,0.000903,0.009031,C25_Krt8_ADI_2
498,Uba1,3.407791,0.809803,0.000884,0.008877,C25_Krt8_ADI_2
499,Pcyox1,3.405408,1.973412,0.000906,0.009041,C25_Krt8_ADI_2


In [20]:
cluster_marker.reset_index(drop = True,inplace=True) 
# drop = True, Do not try to insert index into dataframe columns

In [21]:
cluster_marker.to_csv(result_folder+"EPITHELIAL_WNT_PCLS_cluster_marker_vertically.csv")